In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import time
from typing import List, Tuple, Optional, Union

class SelectedAttention(nn.Module):
    """
    Description:
      Реализация механизма выборочного внимания (Selected Attention) из метода NSA.

    Выборочное внимание работает в несколько этапов:
    1. Сжимает блоки токенов как в CompressedAttention
    2. Вычисляет оценки важности для каждого блока (p_t^slc)
    3. Выбирает топ-n блоков с наивысшими оценками (I_t)
    4. Извлекает оригинальные токены из выбранных блоков
    5. Вычисляет внимание только на этих выбранных токенах

    Параметры:
        hidden_size (int): Размер скрытого состояния
        block_size (int): Размер блока для сжатия (параметр l в статье)
        stride (int): Шаг между блоками (параметр d в статье)
        num_heads (int): Количество голов внимания
        num_selected_blocks (int): Количество блоков для выбора (параметр n в статье)
        dropout (float): Вероятность дропаута
    """
    def __init__(
        self,
        hidden_size: int,
        block_size: int = 32,
        stride: int = 16,
        num_heads: int = 4,
        num_selected_blocks: int = 4,
        dropout: float = 0.1
    ):
        super(SelectedAttention, self).__init__()

        self.hidden_size = hidden_size
        self.block_size = block_size
        self.stride = stride
        self.num_heads = num_heads
        self.head_dim = hidden_size // num_heads
        self.num_selected_blocks = num_selected_blocks

        # Проекции для запросов, ключей и значений
        self.q_proj = nn.Linear(hidden_size, hidden_size)
        self.k_proj = nn.Linear(hidden_size, hidden_size)
        self.v_proj = nn.Linear(hidden_size, hidden_size)

        # Проекция для выхода
        self.out_proj = nn.Linear(hidden_size, hidden_size)

        # Функция сжатия φ (MLP для сжатия блоков)
        self.block_compressor = nn.Sequential(
            nn.Linear(block_size * self.head_dim, 2 * self.head_dim),
            nn.GELU(),
            nn.Linear(2 * self.head_dim, self.head_dim)
        )

        self.dropout = nn.Dropout(dropout)
        self.scale = self.head_dim ** -0.5

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        output_attentions: bool = False
    ) -> Union[torch.Tensor, Tuple[torch.Tensor, List, List]]:
        """
        Description:
          Выполняет выборочное внимание над входной последовательностью.

        Аргументы:
            hidden_states: тензор формы (batch_size, seq_len, hidden_size)
            attention_mask: маска внимания
            output_attentions: флаг для вывода матрицы внимания

        Возвращает:
            output: тензор выхода формы (batch_size, seq_len, hidden_size)
            attention_weights (опционально): веса внимания
            selection_info (опционально): информация о выбранных блоках
        """
        batch_size, seq_len, _ = hidden_states.shape
        device = hidden_states.device

        # Шаг 1: Проекции запросов, ключей и значений
        q = self.q_proj(hidden_states)  # (batch_size, seq_len, hidden_size)
        k = self.k_proj(hidden_states)  # (batch_size, seq_len, hidden_size)
        v = self.v_proj(hidden_states)  # (batch_size, seq_len, hidden_size)

        # Разделение на головы внимания
        q = q.view(batch_size, seq_len, self.num_heads, self.head_dim).permute(0, 2, 1, 3)  # (batch_size, num_heads, seq_len, head_dim)
        k = k.view(batch_size, seq_len, self.num_heads, self.head_dim).permute(0, 2, 1, 3)  # (batch_size, num_heads, seq_len, head_dim)
        v = v.view(batch_size, seq_len, self.num_heads, self.head_dim).permute(0, 2, 1, 3)  # (batch_size, num_heads, seq_len, head_dim)

        # Результаты для всех голов внимания
        context_layers = []
        attention_weights = []
        selection_info = []

        for h in range(self.num_heads):
            # Шаг 2: Разбиение на блоки и сжатие
            blocks_k, block_indices = self._get_blocks(k[:, h])  # Получаем блоки ключей
            blocks_v, _ = self._get_blocks(v[:, h])              # Получаем блоки значений

            # Сжимаем блоки ключей с помощью MLP
            compressed_k = self._compress_blocks(blocks_k)       # (batch_size, num_blocks, head_dim)

            # Шаг 3: Выбор важных блоков на основе сходства с запросом
            # Формула: p_t^slc = Softmax(q_t^T * K_t^cmp)
            scores = torch.matmul(q[:, h], compressed_k.transpose(-1, -2)) * self.scale        # (batch_size, seq_len, num_blocks)
            block_importance = F.softmax(scores, dim=-1)                                       # (batch_size, seq_len, num_blocks)

            # Шаг 4: Выбор блоков с наивысшими оценками
            # Формула: I_t = {i | rank(p_t^slc[i]) <= n}
            num_blocks = len(block_indices)
            num_to_select = min(self.num_selected_blocks, num_blocks)

            # Выбираем топ-k блоков для каждого запроса
            _, selected_block_indices = torch.topk(block_importance, k=num_to_select, dim=-1)  # (batch_size, seq_len, num_to_select)

            # Шаг 5: Извлечение оригинальных токенов из выбранных блоков
            head_context = self._compute_attention_with_selected_blocks(
                q[:, h],                # (batch_size, seq_len, head_dim)
                k[:, h],                # (batch_size, seq_len, head_dim)
                v[:, h],                # (batch_size, seq_len, head_dim)
                block_indices,          # Список диапазонов индексов
                selected_block_indices  # (batch_size, seq_len, num_to_select)
            )

            # Сохраняем результаты
            context_layers.append(head_context)

            if output_attentions:
                attention_weights.append(block_importance)
                selection_info.append((selected_block_indices, block_indices))

        # Объединяем результаты всех голов внимания
        context_layer = torch.stack(context_layers, dim=1)                                    # (batch_size, num_heads, seq_len, head_dim)
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()                        # (batch_size, seq_len, num_heads, head_dim)
        context_layer = context_layer.view(batch_size, seq_len, self.hidden_size)             # (batch_size, seq_len, hidden_size)

        # Финальная проекция
        output = self.out_proj(context_layer)

        if output_attentions:
            return output, attention_weights, selection_info
        else:
            return output

    def _get_blocks(self, x: torch.Tensor) -> Tuple[List[torch.Tensor], List[Tuple[int, int]]]:
        """
        Description:
          Разбивает последовательность на блоки с заданным размером и шагом.

        Аргументы:
            x: тензор формы (batch_size, seq_len, head_dim)

        Возвращает:
            blocks: список блоков
            block_indices: список диапазонов индексов для каждого блока
        """
        batch_size, seq_len, head_dim = x.shape
        blocks = []
        block_indices = []

        for i in range(0, seq_len - self.block_size + 1, self.stride):
            block = x[:, i:i+self.block_size, :]  # (batch_size, block_size, head_dim)
            blocks.append(block)
            block_indices.append((i, i+self.block_size))

        return blocks, block_indices

    def _compress_blocks(self, blocks: List[torch.Tensor]) -> torch.Tensor:
        """
        Description:
          Сжимает блоки токенов в единые представления с помощью MLP.

        Аргументы:
            blocks: список блоков формы (batch_size, block_size, head_dim)

        Возвращает:
            compressed_blocks: тензор формы (batch_size, num_blocks, head_dim)
        """
        batch_size = blocks[0].shape[0]
        num_blocks = len(blocks)

        # Объединяем все блоки в один тензор
        blocks_tensor = torch.cat([block.unsqueeze(1) for block in blocks], dim=1)     # (batch_size, num_blocks, block_size, head_dim)

        # Решейп для передачи в MLP
        blocks_tensor = blocks_tensor.reshape(batch_size * num_blocks, -1)             # (batch_size * num_blocks, block_size * head_dim)

        # Применяем сжатие (функция φ из статьи)
        compressed = self.block_compressor(blocks_tensor)                              # (batch_size * num_blocks, head_dim)

        # Возвращаем к нужной форме
        compressed_blocks = compressed.reshape(batch_size, num_blocks, self.head_dim)  # (batch_size, num_blocks, head_dim)

        return compressed_blocks

    def _compute_attention_with_selected_blocks(
        self,
        queries: torch.Tensor,                # (batch_size, seq_len, head_dim)
        keys: torch.Tensor,                   # (batch_size, seq_len, head_dim)
        values: torch.Tensor,                 # (batch_size, seq_len, head_dim)
        block_indices: List[Tuple[int, int]],
        selected_block_indices: torch.Tensor  # (batch_size, seq_len, num_selected)
    ) -> torch.Tensor:
        """
        Description:
          Вычисляет внимание для каждого запроса, используя только выбранные блоки.

        Аргументы:
            queries: тензор запросов
            keys: тензор ключей
            values: тензор значений
            block_indices: список диапазонов индексов блоков
            selected_block_indices: индексы выбранных блоков для каждого запроса

        Возвращает:
            context: выход внимания для данной головы
        """
        batch_size, seq_len, head_dim = queries.shape
        num_selected = selected_block_indices.size(-1)
        device = queries.device

        # Создаем тензор для результатов
        context = torch.zeros(batch_size, seq_len, head_dim, device=device)

        # Для каждого примера в батче
        for b in range(batch_size):
            # Для каждого запроса
            for q_idx in range(seq_len):
                # Получаем индексы выбранных блоков для данного запроса
                block_idx_list = selected_block_indices[b, q_idx].tolist()

                # Получаем все индексы токенов из выбранных блоков
                token_indices = []
                for block_idx in block_idx_list:
                    if block_idx < len(block_indices):
                        start, end = block_indices[block_idx]
                        # Проверяем, что индексы в пределах последовательности
                        if start < seq_len and end <= seq_len:
                            token_indices.extend(list(range(start, end)))

                # Если список пуст, пропускаем этот запрос
                if not token_indices:
                    continue

                # Убираем дубликаты и сортируем
                token_indices = sorted(set(token_indices))

                # Получаем соответствующие ключи и значения
                q = queries[b, q_idx].unsqueeze(0)        # (1, head_dim)
                k_selected = keys[b, token_indices, :]    # (num_tokens, head_dim)
                v_selected = values[b, token_indices, :]  # (num_tokens, head_dim)

                # Вычисляем внимание
                attention_scores = torch.matmul(q, k_selected.transpose(0, 1)) * self.scale  # (1, num_tokens)
                attention_weights = F.softmax(attention_scores, dim=-1)                      # (1, num_tokens)
                attention_weights = self.dropout(attention_weights)

                # Вычисляем взвешенную сумму
                context[b, q_idx] = torch.matmul(attention_weights, v_selected).squeeze(0)   # (head_dim)

        return context


def demonstrate_selected_attention(use_long_sequence=False):
    """
    Description:
      Демонстрирует работу механизма выборочного внимания и его эффективность.

    Аргументы:
        use_long_sequence: если True, использует последовательность длиной 32K токенов
    """
    # Параметры для демонстрации
    hidden_size = 64
    num_heads = 1            # Для наглядности используем одну голову
    batch_size = 1
    num_selected_blocks = 4  # Количество выбираемых блоков

    if use_long_sequence:
        seq_len = 32000
        block_size = 256
        stride = 128
    else:
        seq_len = 16000
        block_size = 128
        stride = 64

    print("\n" + "="*80)
    print("ДЕМОНСТРАЦИЯ МЕХАНИЗМА ВЫБОРОЧНОГО ВНИМАНИЯ (SELECTED ATTENTION)")
    print("="*80 + "\n")

    print(f"📌 Инициализация модели с параметрами:")
    print(f"  - Размер скрытого состояния: {hidden_size}")
    print(f"  - Размер блока: {block_size}")
    print(f"  - Шаг: {stride}")
    print(f"  - Количество голов внимания: {num_heads}")
    print(f"  - Количество выбираемых блоков: {num_selected_blocks}")
    print(f"  - Длина последовательности: {seq_len}\n")

    # Создаем модель
    model = SelectedAttention(
        hidden_size=hidden_size,
        block_size=block_size,
        stride=stride,
        num_heads=num_heads,
        num_selected_blocks=num_selected_blocks
    )

    # Создаем входные данные с определенными паттернами
    print(f"📌 Создание тестовых данных с паттернами важности...")

    # Для воспроизводимости
    torch.manual_seed(42)

    # Базовый входной тензор
    x = torch.zeros(batch_size, seq_len, hidden_size)

    # Заполняем шумом (по частям для экономии памяти)
    chunk_size = 1000 if use_long_sequence else seq_len
    for i in range(0, seq_len, chunk_size):
        end = min(i + chunk_size, seq_len)
        x[:, i:end, :] = torch.randn(batch_size, end-i, hidden_size) * 0.1

    # Добавляем "важные" токены через равные промежутки
    important_interval = 1000 if use_long_sequence else 16
    for pos in range(0, seq_len, important_interval):
        if pos < seq_len:
            x[:, pos, :] = torch.ones(hidden_size)

    # Добавляем кластер важных токенов в середине
    middle_start = seq_len // 3
    cluster_positions = [(middle_start, middle_start + 20)]

    for start, end in cluster_positions:
        for pos in range(start, min(end, seq_len)):
            x[:, pos, :] = torch.ones(hidden_size) * 0.8

    # Вычисляем важность токенов (сумма значений)
    token_importance = x.sum(dim=2).squeeze().cpu().numpy()

    # Показываем фрагмент важности токенов
    print(f"📌 Важность токенов (фрагмент):")
    start_idx = middle_start - 8
    end_idx = min(middle_start + 28, seq_len)
    for i in range(start_idx, end_idx, 4):
        end = min(i + 4, end_idx)
        values = [f"{token_importance[j]:4.1f}" for j in range(i, end)]
        print(f"  Позиции {i:3d}-{end-1:3d}: {' '.join(values)}")
    print()

    # Выполняем прямой проход модели
    print(f"📌 Выполнение прямого прохода модели...")
    output, attention_weights, selection_info = model(x, output_attentions=True)

    # Анализируем выбранные блоки
    selected_indices, block_indices = selection_info[0]   # Берем результаты первой головы
    block_importance = attention_weights[0][0].detach().cpu().numpy()  # Значимость блоков

    print(f"📌 Анализ результатов выборочного внимания:")

    # Выбираем запрос из кластера важных токенов для анализа
    query_idx = middle_start + 10

    print(f"\n  Анализ для запроса в позиции {query_idx} (внутри кластера важных токенов):")

    # Получаем выбранные блоки для этого запроса
    selected_blocks = selected_indices[0, query_idx].cpu().numpy()

    print(f"  Выбранные блоки для запроса {query_idx}:")
    for i, block_idx in enumerate(selected_blocks):
        start, end = block_indices[block_idx]
        importance = block_importance[query_idx, block_idx]
        print(f"    {i+1}. Блок {block_idx} (позиции {start}-{end}): важность = {importance:.4f}")

    # Показываем распределение важности для всех блоков
    num_blocks = len(block_indices)

    # Создаем фигуру для визуализации
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

    # Визуализация 1: Важность блоков для выбранного запроса
    block_importances = block_importance[query_idx]
    color_map = ['lightgray'] * num_blocks
    for idx in selected_blocks:
        color_map[idx] = 'blue'

    ax1.bar(range(num_blocks), block_importances, color=color_map)
    ax1.set_title(f'Важность блоков для запроса в позиции {query_idx}')
    ax1.set_xlabel('Индекс блока')
    ax1.set_ylabel('Значимость блока')

    # Добавляем порог выбора
    sorted_importances = sorted(block_importances, reverse=True)
    threshold = sorted_importances[min(num_selected_blocks, len(sorted_importances)-1)]
    ax1.axhline(y=threshold, color='red', linestyle='--',
               label=f'Порог выбора ({num_selected_blocks} блоков)')
    ax1.legend()

    # Визуализация 2: Расположение выбранных блоков относительно важности токенов
    ax2.plot(range(seq_len), token_importance, color='gray', alpha=0.7, label='Важность токенов')

    # Выделяем выбранные блоки
    for block_idx in selected_blocks:
        start, end = block_indices[block_idx]
        ax2.axvspan(start, end, color='blue', alpha=0.3)
        ax2.text(start + (end-start)/2, max(token_importance)*0.9,
                f'Блок {block_idx}', ha='center', va='center',
                bbox=dict(facecolor='white', alpha=0.7))

    # Выделяем текущий запрос
    ax2.axvline(x=query_idx, color='red', linestyle='-', label='Текущий запрос')

    ax2.set_title('Расположение выбранных блоков относительно важности токенов')
    ax2.set_xlabel('Позиция в последовательности')
    ax2.set_ylabel('Важность токена')
    ax2.legend()

    plt.tight_layout()

    # Сравнение вычислительной сложности
    print(f"\n📌 Сравнение вычислительной сложности:")

    # Стандартное внимание: O(seq_len^2)
    standard_complexity = seq_len * seq_len

    # Сжатое внимание: O(seq_len * num_blocks)
    compressed_complexity = seq_len * num_blocks

    # Выборочное внимание: O(seq_len * num_selected_blocks * block_size)
    selected_complexity = seq_len * num_selected_blocks * block_size

    print(f"  - Стандартное внимание: O(seq_len^2) = {standard_complexity:,}")
    print(f"  - Сжатое внимание: O(seq_len * num_blocks) = {compressed_complexity:,}")
    print(f"  - Выборочное внимание: O(seq_len * num_selected_blocks * block_size) = {selected_complexity:,}")
    print(f"  - Ускорение относительно стандартного внимания: {standard_complexity / selected_complexity:.2f}x")
    print(f"  - Ускорение относительно сжатого внимания: {compressed_complexity / selected_complexity:.2f}x")

    # Заключение
    print("\n📌 Заключение:")
    print("  - Механизм выборочного внимания успешно идентифицирует и выбирает важные блоки")
    print(f"  - Из {num_blocks} доступных блоков выбираются только {num_selected_blocks} наиболее релевантных")
    print("  - Это значительно сокращает вычислительную сложность при сохранении важной информации")
    print("  - Выборочное внимание позволяет модели фокусироваться на наиболее важных частях контекста")
    print(f"  - При увеличении длины последовательности эффективность только возрастает")

    return fig

In [5]:
# Запускаем демонстрацию на короткой последовательности
fig = demonstrate_selected_attention(use_long_sequence=False)
plt.savefig('selected_attention_visualization.png')
print("\nВизуализация сохранена в файл 'selected_attention_visualization.png'")

plt.close(fig)

run_long_test = input("\nХотите запустить тест на длинной последовательности (32K токенов)? (y/n): ")

if run_long_test.lower() == 'y':
    print("\nЗапуск теста на длинной последовательности. Это может занять некоторое время...")
    try:
        long_fig = demonstrate_selected_attention(use_long_sequence=True)
        plt.savefig('selected_attention_long.png')
        print("\nВизуализация для длинной последовательности сохранена в файл 'selected_attention_long.png'")
        plt.close(long_fig)
    except Exception as e:
        print(f"\nПроизошла ошибка при обработке длинной последовательности: {e}")
        print("Возможно, не хватает памяти для обработки такой длинной последовательности.")
else:
    print("\nТест на длинной последовательности пропущен.")


ДЕМОНСТРАЦИЯ МЕХАНИЗМА ВЫБОРОЧНОГО ВНИМАНИЯ (SELECTED ATTENTION)

📌 Инициализация модели с параметрами:
  - Размер скрытого состояния: 64
  - Размер блока: 128
  - Шаг: 64
  - Количество голов внимания: 1
  - Количество выбираемых блоков: 4
  - Длина последовательности: 16000

📌 Создание тестовых данных с паттернами важности...
📌 Важность токенов (фрагмент):
  Позиции 5325-5328: -1.0  0.3 -1.1 64.0
  Позиции 5329-5332: -1.3 -0.3  0.6  0.4
  Позиции 5333-5336: 51.2 51.2 51.2 51.2
  Позиции 5337-5340: 51.2 51.2 51.2 51.2
  Позиции 5341-5344: 51.2 51.2 51.2 51.2
  Позиции 5345-5348: 51.2 51.2 51.2 51.2
  Позиции 5349-5352: 51.2 51.2 51.2 51.2
  Позиции 5353-5356:  0.2 -0.1 -0.7  0.3
  Позиции 5357-5360:  0.9  0.7  1.1 64.0

📌 Выполнение прямого прохода модели...
📌 Анализ результатов выборочного внимания:

  Анализ для запроса в позиции 5343 (внутри кластера важных токенов):
  Выбранные блоки для запроса 5343:
    1. Блок 7 (позиции 448-576): важность = 0.0041
    2. Блок 95 (позиции 6080